In [20]:
import os

In [21]:
os.chdir("../")

In [22]:
pwd

'd:\\'

In [26]:
os.chdir("/image_classification")


In [27]:
pwd

'd:\\image_classification'

In [28]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    validation_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



In [29]:
from src.image_classification.constants import *
from src.image_classification.utils import read_yaml, create_directories
import tensorflow as tf

In [34]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "train")
        validation_data = os.path.join(self.config.data_ingestion.unzip_dir,"validation")
        create_directories([ 
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            validation_data=Path(validation_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [35]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [38]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True
                                   )
        test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                    rescale = 1./255
                                )
        
        self.training_set = train_datagen.flow_from_directory(self.config.training_data,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
        
        self.test_set = test_datagen.flow_from_directory(self.config.validation_data,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

        
        

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self):
        self.length_of_test_set = len(self.training_set)
        self.length_of_validation_set = len(self.test_set)
        self.steps_per_epoch = self.length_of_test_set
        self.validation_steps = self.length_of_validation_set

        self.model.fit(
            self.training_set,
            validation_data=self.test_set,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [39]:
try:
    config = ConfigurationManager()
    
    

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

Found 337 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Epoch 1/10
11/11 [==============================] - 54s 5s/step - loss: 1.8754 - accuracy: 0.6558 - val_loss: 0.5289 - val_accuracy: 0.8475
Epoch 2/10
11/11 [==============================] - 65s 6s/step - loss: 0.4482 - accuracy: 0.8338 - val_loss: 1.0181 - val_accuracy: 0.7288
Epoch 3/10
11/11 [==============================] - 71s 7s/step - loss: 0.2675 - accuracy: 0.8991 - val_loss: 0.8439 - val_accuracy: 0.8136
Epoch 4/10
11/11 [==============================] - 67s 6s/step - loss: 0.2617 - accuracy: 0.9050 - val_loss: 0.8455 - val_accuracy: 0.8136
Epoch 5/10
11/11 [==============================] - 66s 6s/step - loss: 0.1943 - accuracy: 0.9288 - val_loss: 0.8328 - val_accuracy: 0.7966
Epoch 6/10
11/11 [==============================] - 65s 6s/step - loss: 0.1551 - accuracy: 0.9496 - val_loss: 0.8909 - val_accuracy: 0.7797
Epoch 7/10
11/11 [==============================] - 65s 6s/step - loss: 0.0694 